In [1]:
get_ipython().magic(u'pwd')
get_ipython().magic(u'cd "C:/Users/Wayne/Dropbox/Acer Laptop Sync/Data Science/Facebook Miner using New API"')

import os
import glob
from glob import glob as gg

from collections import defaultdict

import re
import sqlite3

import pandas as pd
from pandas import DataFrame
from pandas import Series

import numpy as np
from numpy import asarray, linspace, corrcoef
from datetime import datetime, date, time

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',300)

C:\Users\Wayne\Dropbox\Acer Laptop Sync\Data Science\Facebook Miner using New API


In [6]:
keywords_9b = pd.read_excel('Network Measures_OC/official_blue_2014_9.xlsx',header=0)
keywords_9y = pd.read_excel('Network Measures_OC/official_yellow_2014_9.xlsx',header=0)
keywords_10b = pd.read_excel('Network Measures_OC/official_blue_2014_10.xlsx',header=0)
keywords_10y = pd.read_excel('Network Measures_OC/official_yellow_2014_10.xlsx',header=0)
keywords_11b = pd.read_excel('Network Measures_OC/official_blue_2014_11.xlsx',header=0)
keywords_11y = pd.read_excel('Network Measures_OC/official_yellow_2014_11.xlsx',header=0)
keywords_12b = pd.read_excel('Network Measures_OC/official_blue_2014_12.xlsx',header=0)
keywords_12y = pd.read_excel('Network Measures_OC/official_yellow_2014_12.xlsx',header=0)

keywords_9b_top500 = keywords_9b.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_9y_top500 = keywords_9y.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_10b_top500 = keywords_10b.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_10y_top500 = keywords_10y.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_11b_top500 = keywords_11b.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_11y_top500 = keywords_11y.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_12b_top500 = keywords_12b.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])
keywords_12y_top500 = keywords_12y.sort("betweenesscentrality", ascending=False)[0:500].set_index(['betweenesscentrality'])

keywords_9b_top500.head(5)

,id,label,timeset,eccentricity,closnesscentrality,harmonicclosnesscentrality,clustering,triangles,eigencentrality
betweenesscentrality,,,,,,,,,
0.148327,香港,香港,NaN,3,0.610184,0.683313,0.016711,113615,1.000000
0.052284,學生,學生,NaN,4,0.548160,0.597183,0.033244,68667,0.688469
0.037603,港人,港人,NaN,3,0.561039,0.613471,0.031434,84050,0.806497
0.037231,大家,大家,NaN,3,0.546659,0.591732,0.038897,69877,0.705981
0.028516,政治,政治,NaN,4,0.536811,0.579894,0.043981,64114,0.650242


In [7]:
con = sqlite3.connect("OccupyHK.sqlite")
cur = con.cursor()
df_9 = pd.read_sql_query("SELECT * FROM Posts WHERE published_date BETWEEN '2014-09-01' AND '2014-09-30'", con)
df_10 = pd.read_sql_query("SELECT * FROM Posts WHERE published_date BETWEEN '2014-10-01' AND '2014-10-31'", con)
df_11 = pd.read_sql_query("SELECT * FROM Posts WHERE published_date BETWEEN '2014-11-01' AND '2014-11-30'", con)
df_12 = pd.read_sql_query("SELECT * FROM Posts WHERE published_date BETWEEN '2014-12-01' AND '2014-12-30'", con)
con.close()

def getyellowposts(df):
    df_yellow = df.loc[df['org_name'].isin(['學民思潮 Scholarism','Occupy Central 佔領中環', '讓愛與和平佔領中環 Occupy Central with Love and Peace' , 
                                            '926 平民在政總現場','佔領期間 民間智慧選輯', '傘下爸媽',
                                             '香港天主教正義和平委員會','Hong Kong Democracy Now', '踢爆五毛','香港警權關注組',
                                             '維護香港核心價值', '我係香港人', '香港人 Secrets','保衛香港自由聯盟','9反佔中 HK oic',
                                             'Fluid Occupiers'])] 
    return df_yellow


def getblueposts(df):
    df_blue = df.loc[df['org_name'].isin(['幫港出聲 Silent Majority','Speak Out HK 港人講地','藍絲帶運動 The Blue Ribbon Movement','向香港警察致敬',
                                               '反佔中苦主大聯盟','泛民不代表我','聲討黃之鋒',
                                               '一人一Like救學生，反對中學生衝擊警察及佔中！',
                                               '「保普選 反佔中」簽左名揮手區！',
                                               '我要真普洱行動','我撐警務處長曾偉雄！',
                                               '我係大學生，我愛國愛港'])] 
    return df_blue


df_9y = getyellowposts(df_9)
df_9b = getblueposts(df_9)
df_10y = getyellowposts(df_10)
df_10b = getblueposts(df_10)
df_11y = getyellowposts(df_11)
df_11b = getblueposts(df_11)
df_12y = getyellowposts(df_12)
df_12b = getblueposts(df_12)

print ("There are",len(df_9y), "OFFICIAL posts sent by anti-establishment civil groups in Sept.")
print ("There are",len(df_9b), "UNOFFICIAL posts sent by pro-establishment civil groups in Sept.")
print ("There are",len(df_10y), "OFFICIAL posts sent by anti-establishment civil groups in Oct.")
print ("There are",len(df_10b), "UNOFFICIAL posts sent by pro-establishment civil groups in Oct.")
print ("There are",len(df_11y), "OFFICIAL posts sent by anti-establishment civil groups in Nov.")
print ("There are",len(df_11b), "UNOFFICIAL posts sent by pro-establishment civil groups in Nov.")
print ("There are",len(df_12y), "OFFICIAL posts sent by anti-establishment civil groups in Dec.")
print ("There are",len(df_12b), "UNOFFICIAL posts sent by pro-establishment civil groups in Dec.")

There are 6539 OFFICIAL posts sent by anti-establishment civil groups in Sept.
There are 1074 UNOFFICIAL posts sent by pro-establishment civil groups in Sept.
There are 9297 OFFICIAL posts sent by anti-establishment civil groups in Oct.
There are 2497 UNOFFICIAL posts sent by pro-establishment civil groups in Oct.
There are 3935 OFFICIAL posts sent by anti-establishment civil groups in Nov.
There are 1638 UNOFFICIAL posts sent by pro-establishment civil groups in Nov.
There are 3034 OFFICIAL posts sent by anti-establishment civil groups in Dec.
There are 1033 UNOFFICIAL posts sent by pro-establishment civil groups in Dec.


In [9]:
def convert2corpus(df):
    corpus = []
    for index, i in df.iterrows():
        corpus.append(i['content'])
    corpus = str(corpus)
    corpus = corpus.replace("[" and "]", " ")
    corpus = corpus.replace("\\", " ")
    return corpus

def cnpuncttoen(corpus):
    #corpus_new = corpus.replace(".", "")
    corpus_new = corpus.replace("。", "*.")
    corpus_new = corpus_new.replace("？", "*?")
    corpus_new = corpus_new.replace(",", "*,")
    corpus_new = corpus_new.replace("！", "*!")
    return corpus_new

def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('*.'):
        if '*?' in s_str:
            list_ret.extend(s_str.split('*?'))
        elif '*!' in s_str:
            list_ret.extend(s_str.split('*!'))
        else:
            list_ret.append(s_str)
    return list_ret

corpus_9y = sentence_split(cnpuncttoen(convert2corpus(df_9y)))
corpus_9b = sentence_split(cnpuncttoen(convert2corpus(df_9b)))
corpus_10y = sentence_split(cnpuncttoen(convert2corpus(df_10y)))
corpus_10b = sentence_split(cnpuncttoen(convert2corpus(df_10b)))
corpus_11y = sentence_split(cnpuncttoen(convert2corpus(df_11y)))
corpus_11b = sentence_split(cnpuncttoen(convert2corpus(df_11b)))
corpus_12y = sentence_split(cnpuncttoen(convert2corpus(df_12y)))
corpus_12b = sentence_split(cnpuncttoen(convert2corpus(df_12b)))

In [24]:
import random
#keywords_9y['example'] = ""

def getexample(row):
    contain_keyword = []
    contain_keyword = [x for x in corpus_12b if row['label'] in x] #remember to change dataframe name
    random.sample(contain_keyword, int(len(contain_keyword)*.1))
    contain_keyword_str = '\n'.join(contain_keyword)
    return contain_keyword_str 

keywords_12b['example'] = keywords_12b.apply(lambda row:  getexample(row), axis=1) 
#for index, row in keywords_9y[102:105].iterrows():
#    contain_keyword = [x for x in corpus_9y if row['label'] in x]
#    #random.sample(contain_keyword, 10)
#    row['example'] = "yes"
keywords_12b[0:10]

,id,label,timeset,eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,componentnumber,modularity_class,clustering,triangles,example
0,屋仔,屋仔,NaN,5,0.334173,0.340270,0.000000,0,30,1.000000,28,對於追求理想的文青，他們對物質的要求不一定很高，未必介意吃「頹飯」、穿舊衣，生命的意義比物質更重要，「四仔主義」（車仔、屋仔、老婆仔、囝囝）對他們來說已過時
1,劇學民,劇學民,NaN,5,0.343915,0.352245,0.000000,0,0,1.000000,10,"http://hk.on.cc/hk/bkn/cnt/news/20141205/bkn-20141205223008224-1205_00822_001.html?eventsection=hk_news&eventid=4028828d47d3fe7e01482b1c75912433'*, '黃之鋒結束絕食鬧劇學民思潮剛宣布，其召集人黃之鋒結束了逾百小時的絕食行動，為五名絕食成員中第二位結束的"
2,感覺,感覺,NaN,4,0.452638,0.470335,0.001278,0,30,0.213140,2446,我只說個人感覺\n但最後的結果，不理犯法者自己感覺如何良好，他們始終逃避不了要面對法律責任\n在這時候，他們卻把司機位讓了給學生瘋狂駕駛，自己又沒有勇氣及早呼籲群眾跳離這失控的列車，任由大家自我感覺良好地留在車上，直至失事流血\n在這時候，他們卻把司機位讓了給學生瘋狂駕駛，自己又沒有勇氣及早呼籲群眾跳離這失控的列車，任由大家自我感覺良好地留在車上，直至失事流血\n由此看來，一般人根本不會感覺得到立法會帶來甚麼問題，亦不會是某些人士所謂的「惡法」\n快啲like《港人講地》Facebook專頁同大家一齊討論啦*!https://www.facebook.com/speakout.hk'*...
3,健民,健民,NaN,4,0.456852,0.473731,0.000429,0,0,0.255759,2820,至於外部勢力介入，陳健民在十二月十八日的電台節目中強調佔中沒有收取外國勢力資金\n請問陳健民：1.佔中是用民主發展網絡的銀行戶口來籌款，戶口號碼是公開的，任何人包括外國勢力都可以隨便存錢進去這個籌款戶口\n陳健民如何確定沒有外國勢力資金在內呢\n*****************************************中大設委會 商陳健民去留星島日報 2014-12-05 新聞佔中三子前日於中區警署自首，承認參與未經批准集會，昨天新增三人自首，令自首人數增至六十八人，當中有立法會議員、醫生、教師、社工等，一旦被定罪，隨時喪失其職位或議席\n佔中三子之中，戴耀廷是港大法律系副教...
4,學民義工,學民義工,NaN,5,0.382070,0.398976,0.000000,0,1,1.000000,300,"It is great !!!""*, '學聯常秘鍾耀華及學民義工蔡思宇身處龍和道前線*,指揮衝擊行動，可恥的雙學'*, '【和平示威 手無寸鐡】海富天橋電梯疑受破壞 u3000匙孔電梯被塞英泥 樓梯淋滑油金鐘海富天橋多次被示威者以不同障礙物，阻塞前往政府總部的道路，兩條扶手電梯已無法運作，其中一條被鐵馬陣堵塞去路"
5,大吉,大吉,NaN,5,0.313478,0.318728,0.000000,0,1,1.000000,1,"」並已有「關門大吉」的心理準備，現時唯有做得一日得一日\n上述例子，都是假定所有心儀明星都是本區校網，最愛劉華，理所當然甲一填劉華，甲二、三均留空，乙一填劉華，這填法不單是經計算後，入讀機會最高；就是大吉利市統一階段落空，叩門階段也可告訴粉絲會會長，自行階段選劉華，統一階段所有首志願選項都只愛劉華，希望可顯示誠意，至少換來面試機會\nhttps://www.facebook.com/speakout.hk'*, '【精選文章】小一入學報告書值得參考梁永樂報章前教育新聞組長 大吉利市，絕大多數在自行分配學位落空的家長，相信極少人成功在這個階段取得學位"
6,幽默,幽默,NaN,5,0.337587,0.343926,0.000000,0,1,1.000000,45,是次辯論為第一次初賽，過程極之精彩，同學清晰表達自己的論點，同時幽默地引出多個生活化的例子說明，贏得觀眾掌聲
7,物價,物價,NaN,4,0.401120,0.419978,0.000018,0,26,0.472727,468,先旨聲明，筆者不是要貶低這些國家，但很明顯，香港跟肯雅、阿富汗和烏干達等國，在經濟發展、物價、薪酬水平、民情、風俗、習慣、人口、規模等範疇都有很大的差別\n至昨午，盧布兌美元下跌1.3%，兌歐元亦下跌0.2%，盧布幣值於今星期已合共貶值20%，引發俄羅斯民眾對物價飛漲的憂慮，莫斯科不少居民湧往搶購雪櫃、洗衣機和電視等電器，冀在加價前購入，有民眾直言兩周前購入的洗衣機，現時價格已上升了兩成半，而現時再要面對美國加強制裁，分析認為俄羅斯經濟若進一步惡化，一旦出現債務違約，俄國在歐洲及亞洲的貿易夥伴國亦將受影響\n」他表示，當局會根據既定的調整機制，按截至2014年10月底的社會保障援助物價...
8,大城市,大城市,NaN,4,0.414502,0.432697,0.000024,0,6,0.518717,291,"我們經常見到外國年輕人離鄉背井，憑英語之便到全球各大城市工作，香港也見到不少這類外國人，神卅各大城市更是比比皆是，為何外國的年輕人能，我們兼備中西文化薰陶的香港年輕人不能\n快啲like《港人講地》Facebook專頁發表意見啦*!https://www.facebook.com/speakout.hk'*, '【環境保育】攝影師「除光害」 大城市也可見銀河入夜後的繁華都市燈火璀燦，令人玹目，但也令光污染越來越嚴重，有研究指出，光污染影響人類和作息，對人類健康造成影響，同樣亦影響大自然生態\n不是身歷其境，只要留意一下國際新聞報道，也可以感受到外國大城市的治安是什麼狀\n桂強芳指出，香..."
9,執政黨,執政黨,NaN,5,0.324409,0.329370,0.000000,0,30,1.000000,1,反對黨在新加坡處處受到限制，難以形成挑戰執政黨的力量


In [25]:
keywords_12b.to_pickle('keywords_12b.pkl')
keywords_12b.to_excel('keywords_12b.xlsx')

C:\Users\wayne\Anaconda\envs\py33\lib\site-packages\xlsxwriter\worksheet.py:845: UserWarning: Ignoring URL 'https://www.facebook.com/speakout.hk'*,%20'【精選文章】啟蒙柔性釋法%20重塑一國兩制陳建強香港專業人士協會主席七十九天的佔領噩夢，雖以失敗作結，但已猶如夜半旱天雷，驚醒了長期沉醉於繁華美夢的港人，亦向中央和特區政府發出一響令人驚悸的警號，就是香港不再是只談經濟的政治避風港，反成可任意犧牲經濟的政治黑洞；只要其政治訴求遭拂逆，有人就會「無畏無懼」地走上街頭抗爭，衝擊法治
逗留開羅期間，與當地商戶閒談，得悉開羅的旅遊業一落千丈，昔日繁華景象不再
快啲like《港人講地》Facebook專頁發表意見啦*!https://www.facebook.com/speakout.hk'*,%20'【環境保育】攝影師「除光害」%20大城市也可見銀河入夜後的繁華都市燈火璀燦，令人玹目，但也令光污染越來越嚴重，有研究指出，光污染影響人類和作息，對人類健康造成影響，同樣亦影響大自然生態' > 255 characters since it exceeds Excel's limit for URLS
  "Excel's limit for URLS" % url)
C:\Users\wayne\Anaconda\envs\py33\lib\site-packages\xlsxwriter\worksheet.py:845: UserWarning: Ignoring URL 'https://www.facebook.com/speakout.hk'*,%20'【精選文章】啟蒙柔性釋法%20重塑一國兩制陳建強香港專業人士協會主席七十九天的佔領噩夢，雖以失敗作結，但已猶如夜半旱天雷，驚醒了長期沉醉於繁華美夢的港人，亦向中央和特區政府發出一響令人驚悸的警號，就是香港不再是只談經濟的政治避風港，反成可任意犧牲經濟的政治黑洞；只要其政治訴求遭拂逆，有人就會「無畏無懼」地走上街頭抗爭，衝擊法治
未來，政府和社會應加強針對青少年的基本法的普及教育，讓年輕人認識基本法的歷史和背後的精神，重塑香港人的法治理性，加強對國家歸屬感，擁護「一國兩制」方